In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,nltk

In [3]:
#read the dataset
data = pd.read_excel('/content/drive/MyDrive/consumer/Source/dataset.xlsx')
data

,text,target
0,"আমি যখন তাদের কাছ থেকে এই ল্যাপটপটি কিনছিলাম, ...",1.0
1,তাদের বিক্রয়োত্তর পরিষেবা এত দেরিতে যে তাদের ...,1.0
2,তাদের কাছ থেকে কিছু কেনার আগে তারা খুব মিষ্টি....,1.0
3,"ভাই, এক বছর আগে একটি পিসি বিল্ড করছি। এখন মাদ...",1.0
4,আমার ও M 21 ডিসপ্লেতে সমস্যা। এ খনো ওয়ারেন্টি ...,1.0
...,...,...
3021,হ্যাঁ এই ফটো এবং ভিডিওর জন্য আমার জন্য নিখুঁত ...,0.0
3022,স্বাভাবিক ব্যবহারের জন্য ভাল কী বোর্ড। গেমিং ব...,0.0
3023,স্মার্ট এবং আরামদায়ক। । পুরোন জিনিস খুব ভালো ...,0.0
3024,হয়তো এটির সাথে ফিল্মের অন্তত একটি প্যাক থাকার...,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    3026 non-null   object 
 1   target  3026 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.4+ KB


In [5]:
data.groupby('target').size().reset_index(name='counts')

,target,counts
0,0.0,1217
1,1.0,1809


In [6]:
#Define the function for cleaning the data
def process_text(text): 
    text = re.sub('[^\u0980-\u09FF]',' ',text)
    text = re.sub('[?.`*^()!°¢܌Ͱ̰ߒנ~×Ҡߘ:ҰߑÍ|।;!,&%\'@#$><A-Za-z0+-9=./''""_০-৯]', '', text)
    text = re.sub(r'(\W)(?=\1)', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'ߑͰߑ̰ߒנ', '', text)
    text = re.sub(r'ߎɰߎɰߎɍ', '', text)
    return text

In [7]:
data['cleaned_text'] = data['text'].apply(process_text)  
data

,text,target,cleaned_text
0,"আমি যখন তাদের কাছ থেকে এই ল্যাপটপটি কিনছিলাম, ...",1.0,আমি যখন তাদের কাছ থেকে এই ল্যাপটপটি কিনছিলাম ত...
1,তাদের বিক্রয়োত্তর পরিষেবা এত দেরিতে যে তাদের ...,1.0,তাদের বিক্রয়োত্তর পরিষেবা এত দেরিতে যে তাদের ...
2,তাদের কাছ থেকে কিছু কেনার আগে তারা খুব মিষ্টি....,1.0,তাদের কাছ থেকে কিছু কেনার আগে তারা খুব মিষ্টি ...
3,"ভাই, এক বছর আগে একটি পিসি বিল্ড করছি। এখন মাদ...",1.0,ভাই এক বছর আগে একটি পিসি বিল্ড করছি এখন মাদারব...
4,আমার ও M 21 ডিসপ্লেতে সমস্যা। এ খনো ওয়ারেন্টি ...,1.0,আমার ও ডিসপ্লেতে সমস্যা এ খনো ওয়ারেন্টি আছে প্...
...,...,...,...
3021,হ্যাঁ এই ফটো এবং ভিডিওর জন্য আমার জন্য নিখুঁত ...,0.0,হ্যাঁ এই ফটো এবং ভিডিওর জন্য আমার জন্য নিখুঁত ...
3022,স্বাভাবিক ব্যবহারের জন্য ভাল কী বোর্ড। গেমিং ব...,0.0,স্বাভাবিক ব্যবহারের জন্য ভাল কী বোর্ড গেমিং বা...
3023,স্মার্ট এবং আরামদায়ক। । পুরোন জিনিস খুব ভালো ...,0.0,স্মার্ট এবং আরামদায়ক পুরোন জিনিস খুব ভালো লেগ...
3024,হয়তো এটির সাথে ফিল্মের অন্তত একটি প্যাক থাকার...,0.0,হয়তো এটির সাথে ফিল্মের অন্তত একটি প্যাক থাকার...


In [8]:
stopwords_list ='/content/drive/MyDrive/consumer/Source/stopwords-bn.txt'

In [9]:
# Stopwords Removal

def stopword_removal(Comment,stopwords= stopwords_list):
    stp,num_of_stopwords =stopwords_info(stopwords)
    result = Comment.split()
    Comments = [word.strip() for word in result if word not in stp ]
    Comments =" ".join(Comments)
    return Comments

def stopwords_info(filename):
    stp = open(filename,'r',encoding='utf-8').read().split()
    num_of_stopwords = len(stp)
    return stp,num_of_stopwords
    return text

In [10]:
data['cleaned_text'] = data['cleaned_text'].apply(stopword_removal)  
data

,text,target,cleaned_text
0,"আমি যখন তাদের কাছ থেকে এই ল্যাপটপটি কিনছিলাম, ...",1.0,ল্যাপটপটি কিনছিলাম বলেছিল দ্রুত পরিষেবা দেবে ব...
1,তাদের বিক্রয়োত্তর পরিষেবা এত দেরিতে যে তাদের ...,1.0,বিক্রয়োত্তর পরিষেবা দেরিতে কেনা সবচেয়ে খারাপ...
2,তাদের কাছ থেকে কিছু কেনার আগে তারা খুব মিষ্টি....,1.0,কেনার মিষ্টি আচরণ খারাপ ওঠে
3,"ভাই, এক বছর আগে একটি পিসি বিল্ড করছি। এখন মাদ...",1.0,ভাই এক বছর পিসি বিল্ড করছি মাদারবোর্ডে সমস্যা ...
4,আমার ও M 21 ডিসপ্লেতে সমস্যা। এ খনো ওয়ারেন্টি ...,1.0,ডিসপ্লেতে সমস্যা খনো ওয়ারেন্টি সাত মাস কিনেছি ...
...,...,...,...
3021,হ্যাঁ এই ফটো এবং ভিডিওর জন্য আমার জন্য নিখুঁত ...,0.0,হ্যাঁ ফটো ভিডিওর নিখুঁত ক্যামেরা অল্প দামে পুর...
3022,স্বাভাবিক ব্যবহারের জন্য ভাল কী বোর্ড। গেমিং ব...,0.0,স্বাভাবিক ব্যবহারের ভাল বোর্ড গেমিং পছন্দসই মত...
3023,স্মার্ট এবং আরামদায়ক। । পুরোন জিনিস খুব ভালো ...,0.0,স্মার্ট আরামদায়ক পুরোন জিনিস ভালো লেগেছে
3024,হয়তো এটির সাথে ফিল্মের অন্তত একটি প্যাক থাকার...,0.0,এটির সাথে ফিল্মের প্যাক থাকার স্ট্যান্ডবাই যথে...


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   text          3026 non-null   object 
 1   target        3026 non-null   float64
 2   cleaned_text  3026 non-null   object 
dtypes: float64(1), object(2)
memory usage: 71.0+ KB


In [12]:
data

,text,target,cleaned_text
0,"আমি যখন তাদের কাছ থেকে এই ল্যাপটপটি কিনছিলাম, ...",1.0,ল্যাপটপটি কিনছিলাম বলেছিল দ্রুত পরিষেবা দেবে ব...
1,তাদের বিক্রয়োত্তর পরিষেবা এত দেরিতে যে তাদের ...,1.0,বিক্রয়োত্তর পরিষেবা দেরিতে কেনা সবচেয়ে খারাপ...
2,তাদের কাছ থেকে কিছু কেনার আগে তারা খুব মিষ্টি....,1.0,কেনার মিষ্টি আচরণ খারাপ ওঠে
3,"ভাই, এক বছর আগে একটি পিসি বিল্ড করছি। এখন মাদ...",1.0,ভাই এক বছর পিসি বিল্ড করছি মাদারবোর্ডে সমস্যা ...
4,আমার ও M 21 ডিসপ্লেতে সমস্যা। এ খনো ওয়ারেন্টি ...,1.0,ডিসপ্লেতে সমস্যা খনো ওয়ারেন্টি সাত মাস কিনেছি ...
...,...,...,...
3021,হ্যাঁ এই ফটো এবং ভিডিওর জন্য আমার জন্য নিখুঁত ...,0.0,হ্যাঁ ফটো ভিডিওর নিখুঁত ক্যামেরা অল্প দামে পুর...
3022,স্বাভাবিক ব্যবহারের জন্য ভাল কী বোর্ড। গেমিং ব...,0.0,স্বাভাবিক ব্যবহারের ভাল বোর্ড গেমিং পছন্দসই মত...
3023,স্মার্ট এবং আরামদায়ক। । পুরোন জিনিস খুব ভালো ...,0.0,স্মার্ট আরামদায়ক পুরোন জিনিস ভালো লেগেছে
3024,হয়তো এটির সাথে ফিল্মের অন্তত একটি প্যাক থাকার...,0.0,এটির সাথে ফিল্মের প্যাক থাকার স্ট্যান্ডবাই যথে...


In [13]:
#Save the cleaned data to a new file
data.to_excel('/content/drive/MyDrive/consumer/Source/dataset_clean.xlsx')